# Archivo para intentar correr todo en la infraestructura del curso

## Cruce espacial

In [ ]:
#dbutils.fs.put("/FileStore/my-stuff/my-file.txt", "This is the actual text that will be saved to disk. Like a 'Hello world!' example")

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
from pyspark.sql.functions import year, month, dayofmonth, concat_ws, lpad
from pyspark.sql.functions import col
from pyspark.sql.functions import count, sum
import pandas as pd
#import seaborn as sns
import pyarrow.parquet as pq
#import geopandas




os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.hadoop:hadoop-aws:3.2.2,io.delta:delta-core_2.12:1.1.0  pyspark-shell "
config = {
    "spark.jars.packages":"org.apache.hadoop:hadoop-aws:3.2.2",
    "spark.kubernetes.namespace": "spark",
    "spark.kubernetes.container.image": "cronosnull/abd-spark-base:202301",
    "spark.executor.instances": "4",
    "spark.executor.memory": "1g",
    "spark.executor.cores": "1",
    "spark.driver.port":"38889",
    "spark.driver.blockManager.port":"7777",
    "spark.driver.bindAddress": "0.0.0.0",
    "spark.executor.memory": "4g", #4
    "spark.driver.host":"172.24.99.30",
    "spark.kubernetes.executor.request.cores":"500m",
    "spark.hadoop.fs.s3a.endpoint": "http://172.24.99.18:9000",
    # Credenciales de MinNIO, no olvide asignar las variables de entorno
    "spark.hadoop.fs.s3a.access.key": os.environ.get('MINIO_USERNAME', "--"),
    "spark.hadoop.fs.s3a.secret.key": os.environ.get('MINIO_PASSWORD',"--"),
    "spark.hadoop.fs.s3a.path.style.access": True,
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    "spark.kubernetes.local.dirs.tmpfs":True,

}
def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://172.24.99.68:16443")
    for key, value in config.items():
        conf.set(key, value)    
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()

In [2]:
spark = get_spark_session("grupo03-taxis", SparkConf())

In [3]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 1.8.0_322
Branch 
Compiled by user  on 2022-03-26T09:34:47Z
Revision 
Url 
Type --help for more information.


In [4]:
spark

In [33]:
pip list

Package                 Version    
----------------------- -----------
anyio                   3.5.0      
argon2-cffi             21.3.0     
argon2-cffi-bindings    21.2.0     
attrs                   22.1.0     
backcall                0.2.0      
beautifulsoup4          4.11.1     
bitarray                2.5.1      
bleach                  4.1.0      
branca                  0.6.0      
brotlipy                0.7.0      
certifi                 2023.7.22  
cffi                    1.14.5     
charset-normalizer      2.0.4      
click                   8.0.4      
click-plugins           1.1.1      
cligj                   0.7.2      
conda                   23.1.0     
conda-package-handling  2.0.2      
conda-package-streaming 0.7.0      
contourpy               1.0.5      
cryptography            38.0.4     
cycler                  0.11.0     
decorator               5.1.1      
defusedxml              0.7.1      
entrypoints             0.4        
fastjsonschema          2.16

In [27]:
import requests 
import geopandas as gpd #geopandas 0.9
#import pygeos #0.8
#gpd.options.use_pygeos = True


request = requests.get('https://raw.githubusercontent.com/AlberGonglius/taller-1-big-data/main/NYCTaxiZones.json')
geojson = request.json()
zone_gdf = gpd.GeoDataFrame.from_features(geojson['features'])

In [28]:
#Función para el geojoin 
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, pandas_udf

def create_sjoin_udf(gdf_with_poly,join_column_name):
    def sjoin_settlement(x, y):
        gdf_temp = gpd.GeoDataFrame(data = [[x] for x in range(len(x))],geometry=gpd.points_from_xy(x,y),columns=['id'])
        settlement = gpd.sjoin(gdf_temp,gdf_with_poly[['borough','geometry']],how='left')#.fillna(np.nan)
        
        return settlement.groupby('id').agg({'borough':lambda x: list(x)}).reset_index().sort_values(by='id').loc[:,join_column_name].astype('str').str.strip('[]').str.strip("'")# if pd.isnull(sum(x)) == False else np.nan}).reset_index().sort_values(by='id').loc[:,join_column_name]
    return pandas_udf(sjoin_settlement, returnType=StringType())
sjoin_udf = create_sjoin_udf(zone_gdf,'borough')

In [29]:
zone_gdf

,geometry,shape_area,objectid,shape_leng,location_id,zone,borough
0,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ...",0.0007823067885,1,0.116357453189,1,Newark Airport,EWR
1,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens
2,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ...",0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx
3,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ...",0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan
4,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ...",0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island
...,...,...,...,...,...,...,...
258,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ...",0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn
259,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ...",0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx
260,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ...",0.000422345326907,260,0.133514154636,260,Woodside,Queens
261,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ...",0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan


In [30]:
#se leen los primeros parquet que se saben tienen la misma esrtuctura y solo las cols que importan
#2010 cambia los nombres de las columnas a 'pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount'
#2009 ya está
df1 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-01.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df2 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-02.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df3 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-03.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df4 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-04.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df5 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-05.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df6 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-06.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df7 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-07.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df8 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-08.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df9 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-09.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df10 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-10.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df11 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-11.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')
df12 = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2010-12.parquet").select('pickup_datetime', 'trip_distance',  'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','total_amount')

df = df1.union(df2).union(df3).union(df4).union(df5).union(df6).union(df7).union(df8).union(df9).union(df10).union(df11).union(df12)

df.show()

+-------------------+------------------+------------------+---------------+------------------+----------------+-----------------+
|    pickup_datetime|     trip_distance|  pickup_longitude|pickup_latitude| dropoff_longitude|dropoff_latitude|     total_amount|
+-------------------+------------------+------------------+---------------+------------------+----------------+-----------------+
|2010-01-26 07:41:00|              0.75|        -73.956778|       40.76775|        -73.965957|       40.765232|              5.0|
|2010-01-30 23:31:00|               5.9|-73.99611799999998|      40.763932|-73.98151199999998|       40.741193|             16.3|
|2010-01-18 20:22:20|               4.0|        -73.979673|       40.78379|-73.91785199999998|        40.87856|             12.7|
|2010-01-09 01:18:00|               4.7|        -73.977922|      40.763997|-73.92390799999998|       40.759725|             14.3|
|2010-01-18 19:10:14|0.5999999999999999|        -73.990924|      40.734682|-73.99551099999

In [31]:
pu_lon ='pickup_longitude'
pu_lat = 'pickup_latitude'
do_lon ='dropoff_longitude'
do_lat = 'dropoff_latitude'
dat = 'pickup_datetime'
amt = 'total_amount'
dist = 'trip_distance'

In [32]:
#df_subset = df.limit(50)
#zonas de recogida y dejada
df = df.withColumn('PU_zone',sjoin_udf(df.pickup_longitude,df.pickup_latitude ))
df = df.withColumn('DO_zone',sjoin_udf(df.dropoff_longitude,df.dropoff_latitude))
df.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-28-f6d58a01fa7a>", line 8, in sjoin_settlement
  File "/opt/conda/lib/python3.8/site-packages/geopandas/tools/sjoin.py", line 124, in sjoin
    indices = _geom_predicate_query(left_df, right_df, predicate)
  File "/opt/conda/lib/python3.8/site-packages/geopandas/tools/sjoin.py", line 217, in _geom_predicate_query
    input_geoms = left_df.geometry
  File "/opt/conda/lib/python3.8/site-packages/geopandas/base.py", line 2788, in sindex
    return self.geometry.values.sindex
  File "/opt/conda/lib/python3.8/site-packages/geopandas/array.py", line 295, in sindex
    self._sindex = _get_sindex_class()(self.data)
  File "/opt/conda/lib/python3.8/site-packages/geopandas/sindex.py", line 21, in _get_sindex_class
    raise ImportError(
ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https

Este error no fue posible solucionarlo ni instalando una imágen personalizadacon los paquetes preinstalados. En el listado se observa que rtree si está instalado.

In [ ]:
#crea la col "pickup_date" que contiene el año-mes-dia de "tpep_pickup_datetime"

#se necesita independientemente cada una...
df = df.withColumn("pickup_year", year(df[dat]))
df = df.withColumn("pickup_month", month(df[dat]))
#df = df.withColumn("pickup_day", dayofmonth(df[dat]))


# Muestra el DataFrame resultante con la nueva columna
df.show()

In [ ]:
#Repartición para ver si mejora desempeño
#df = df.repartition(30, 'pickup_day')


In [ ]:
def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]


def toPandas2(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

In [ ]:
from pyspark.sql.functions import count, sum

agg_df = df.groupBy("pickup_year", "pickup_month", "PU_zone", "DO_zone" ).agg(count("*").alias("conteo"), sum("Total_Amt").alias("tot_amt"), sum("Trip_Distance").alias("distancia"))

#si quisiera redondear:
#agg_df = agg_df.withColumn("tot_amt", round(agg_df["tot_amt"], 2))


# Muestra el nuevo DataFrame
#agg_df.show()

In [ ]:
a= agg_df.toPandas()

In [ ]:
display(a)

## Cruce de los demás archivos

In [18]:
#lista para sacar 2011 a 2016. Para generar el resto cambiar el rango a range(2017,2023)
#Se realiza de esta manera porque no fue posible generar el archivo agregado sobre todo el dataset y se particionó de esta manera.
lista = []

for i in range(2011,2017):
    if i <2016:
        for j in range(1,13):
            if j < 10:
                lista.append("s3a://taxis/parquet/yellow_tripdata_"+str(i)+"-0"+str(j)+".parquet")
            else:
                lista.append("s3a://taxis/parquet/yellow_tripdata_"+str(i)+"-"+str(j)+".parquet")
    else:
        for j in range(1,12):
            if j < 10:
                lista.append("s3a://taxis/parquet/yellow_tripdata_"+str(i)+"-0"+str(j)+".parquet")
            else:
                lista.append("s3a://taxis/parquet/yellow_tripdata_"+str(i)+"-"+str(j)+".parquet")

In [19]:
lista

['s3a://taxis/parquet/yellow_tripdata_2011-01.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-02.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-03.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-04.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-05.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-06.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-07.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-08.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-09.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-10.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-11.parquet',
 's3a://taxis/parquet/yellow_tripdata_2011-12.parquet',
 's3a://taxis/parquet/yellow_tripdata_2012-01.parquet',
 's3a://taxis/parquet/yellow_tripdata_2012-02.parquet',
 's3a://taxis/parquet/yellow_tripdata_2012-03.parquet',
 's3a://taxis/parquet/yellow_tripdata_2012-04.parquet',
 's3a://taxis/parquet/yellow_tripdata_2012-05.parquet',
 's3a://taxis/parquet/yellow_tripdata_2012-06.pa

In [20]:
df = spark.read.parquet("s3a://taxis/parquet/yellow_tripdata_2016-12.parquet").select('tpep_pickup_datetime', 'trip_distance',  'PULocationID', 'DOLocationID','total_amount')
df.show()

+--------------------+-------------+------------+------------+------------+
|tpep_pickup_datetime|trip_distance|PULocationID|DOLocationID|total_amount|
+--------------------+-------------+------------+------------+------------+
| 2016-12-01 00:26:26|          0.0|         145|         145|         3.8|
| 2016-12-01 00:08:13|          4.2|         262|         226|        16.8|
| 2016-12-01 00:36:29|          1.1|         238|          75|         6.8|
| 2016-12-01 00:55:28|          1.2|         237|         230|         7.8|
| 2016-12-01 00:13:08|         1.48|         142|         161|       15.38|
| 2016-12-01 00:43:16|         2.25|         142|         262|       12.88|
| 2016-12-01 00:27:18|          3.4|         234|         236|        14.8|
| 2016-12-01 00:35:50|          0.7|         237|         237|        6.95|
| 2016-12-01 00:19:45|          2.2|         162|         107|        10.3|
| 2016-12-01 00:36:40|          1.2|         114|          79|         8.3|
| 2016-12-01

In [21]:
for file_path in lista:
    df1 = spark.read.parquet(file_path).select('tpep_pickup_datetime', 'trip_distance',  'PULocationID', 'DOLocationID','total_amount')
    df=df.union(df1)

In [22]:
df = df.withColumn("pickup_year", year(df["tpep_pickup_datetime"]))
df = df.withColumn("pickup_month", month(df["tpep_pickup_datetime"]))
#df = df.withColumn("pickup_day", dayofmonth(df["tpep_pickup_datetime"]))


In [12]:
taxi_zone =  spark.read.csv("s3a://taxis/taxi_zone_lookup.csv", header = True)

In [13]:
taxi_zone = taxi_zone.withColumn("LocationId", col("LocationId").cast("int"))

In [14]:
taxi_zone

DataFrame[LocationId: int, Borough: string, Zone: string, service_zone: string]

In [23]:

# Selecciona las columnas de df2 que deseas agregar
#Zona
#col_zones = taxi_zone.select("LocationId", "Zone")
#zona de servicio
zone = 'Zone'
col_zones = taxi_zone.select("LocationId", zone)

# Agrega las columnas de PU al DataFrame original
df = df.join(col_zones, df.PULocationID == col_zones.LocationId, how="left")
df = df.withColumnRenamed(zone, "PU_Zone")
#elimina la columna LocationId porque ya no se necesita
df = df.drop("LocationId")
#se hace lo mismo para el punto de llegada
df = df.join(col_zones, df.DOLocationID == col_zones.LocationId, how="left")
df = df.withColumnRenamed(zone, "DO_Zone")
#elimina la columna LocationId porque ya no se necesita
df = df.drop("LocationId")

# Muestra el resultado
df.show()

+--------------------+-------------+------------+------------+------------+-----------+------------+--------------------+--------------------+
|tpep_pickup_datetime|trip_distance|PULocationID|DOLocationID|total_amount|pickup_year|pickup_month|             PU_Zone|             DO_Zone|
+--------------------+-------------+------------+------------+------------+-----------+------------+--------------------+--------------------+
| 2016-12-01 00:26:26|          0.0|         145|         145|         3.8|       2016|          12|Long Island City/...|Long Island City/...|
| 2016-12-01 00:08:13|          4.2|         262|         226|        16.8|       2016|          12|      Yorkville East|           Sunnyside|
| 2016-12-01 00:36:29|          1.1|         238|          75|         6.8|       2016|          12|Upper West Side N...|   East Harlem South|
| 2016-12-01 00:55:28|          1.2|         237|         230|         7.8|       2016|          12|Upper East Side S...|Times Sq/Theatre ...|

In [24]:
from pyspark.sql.functions import count, sum

# Agrupa el DataFrame por cualquier columna (por ejemplo, "col1")
# y realiza las agregaciones de conteo y suma
agg_df = df.groupBy("pickup_year","pickup_month","PU_Zone", "DO_Zone" ).agg(count("*").alias("conteo"), sum("total_amount").alias("tot_amt"), sum("trip_distance").alias("distancia"))

#si quisiera redondear:
#agg_df = agg_df.withColumn("tot_amt", round(agg_df["tot_amt"], 2))


# Muestra el nuevo DataFrame
agg_df.show()

+-----------+------------+--------------------+--------------------+------+------------------+------------------+
|pickup_year|pickup_month|             PU_Zone|             DO_Zone|conteo|           tot_amt|         distancia|
+-----------+------------+--------------------+--------------------+------+------------------+------------------+
|       2011|           1|         Murray Hill| Lincoln Square East|  6747| 78132.66999999972|16231.510000000066|
|       2011|           1|Upper West Side S...|Upper West Side N...| 30904|183989.21000000418|26444.189999999875|
|       2011|           1|      Yorkville West|Upper East Side N...| 19302|111438.52000000021|13589.480000000087|
|       2011|           1|Penn Station/Madi...|      Midtown Center| 25421| 203175.0099999982|33611.819999999985|
|       2011|           1|Greenwich Village...|        Bloomingdale|   338|           7038.62|2179.1599999999985|
|       2011|           1|   Battery Park City|         Boerum Hill|   166| 2468.7199999

In [14]:
agg_df.schema

StructType(List(StructField(pickup_year,IntegerType,true),StructField(pickup_month,IntegerType,true),StructField(PU_Zone,StringType,true),StructField(DO_Zone,StringType,true),StructField(conteo,LongType,false),StructField(tot_amt,DoubleType,true),StructField(distancia,DoubleType,true)))

In [26]:
agg_df.write.parquet('s3a://user-data/grupo-03/taxis-2011')